In [1]:
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
import paramiko
import ftplib
import sqlite3
from pathlib import Path
import pandas as pd


In [2]:
# generate some fake 
n = 2000
x = np.linspace(0.0, 10.0, n)
np.random.seed(0)
y = np.cumsum(np.random.randn(n)*10).astype(int)

In [3]:
fig_hist = plt.figure( title='Histogram')
hist = plt.hist(y, bins=25)

In [4]:
hist.bins = 10;

In [5]:
slider = widgets.IntSlider(description='Bins number', min=1, max=100, v_model=30)

In [6]:
widgets.link((hist, 'bins'), (slider, 'value'))

fig_lines = plt.figure( title='Line Chart')
lines = plt.plot(x, y)

fig_lines.layout.width = 'auto'
fig_lines.layout.height = 'auto'
fig_hist.layout.width = 'auto'
fig_hist.layout.height = 'auto'

grid_layout = widgets.GridspecLayout(5, 3)

grid_layout[:2, :] = fig_lines
grid_layout[2:4, :] = fig_hist
grid_layout[4, 1] = slider

grid_layout.layout.height = '1000px'

grid_layout

GridspecLayout(children=(Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScal…

In [7]:
selector = plt.brush_int_selector()
def update_range(*ignore):
    if selector.selected is not None and len(selector.selected) == 2:
        xmin, xmax = selector.selected
        mask = (x > xmin) & (x < xmax)
        hist.sample = y[mask]
selector.observe(update_range, 'selected')        

In [8]:
from IPython.display import IFrame
IFrame('https://sustainablecharging.nl/', width=1200, height=500)

In [9]:
from pyowm.owm import OWM
from pyowm.utils import timestamps
from datetime import datetime, timedelta, timezone

In [10]:
def get_weather():
    
    owm = OWM('1a4df9d4817c3d16e92b272d59531753')
    mgr = owm.weather_manager()
    one_call = mgr.one_call(lat=51.98787601885725, lon=5.950209138832937)
    #one_call.forecast_daily[0].temperature('celsius').get('feels_like_morn', None) #Ex.: 7.7
    forecast_hourly = one_call.forecast_hourly
    
    nr_of_weathers = len(forecast_hourly)
    Cloud = np.zeros(len(forecast_hourly))
    Temperature = np.zeros(len(forecast_hourly))
    #Wind_speed = np.zeros(len(forecast_hourly))
    #Wind_direction = np.zeros(len(forecast_hourly))
    Humidity = np.zeros(len(forecast_hourly))
    Air_pressure = np.zeros(len(forecast_hourly))

    for i in range(nr_of_weathers):
        dt = (datetime.fromtimestamp(forecast_hourly[i].ref_time)).strftime('%Y-%m-%d %H:%M:%S')
        Cloud[i] = forecast_hourly[i].clouds
        Temperature[i] = forecast_hourly[i].temperature('celsius')['temp']
        #Wind_speed[i] = forecast_hourly[i].wind()['speed']
        #Wind_direction[i] = forecast_hourly[i].wind()['deg']
        Humidity[i] = forecast_hourly[i].humidity
        Air_pressure[i] = forecast_hourly[i].pressure['press']*0.1
        #print(dt, forecast_hourly[i].temperature('celsius'),forecast_hourly[i].clouds )
    return Temperature, Air_pressure , Humidity, Cloud

In [11]:
#Temperature, Air_pressure , Humidity, Cloud = get_weather()

In [12]:
#print('Temperature',Temperature)
#print('Cloud', Cloud)
#print('Humidity', Humidity)
#print('Air_pressure', Air_pressure)

In [28]:
def get_modbus():
    
    host = "80.113.19.27"
    port = 22
    password = "controlsystem"
    username = "pi"

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

    ftp = ssh.open_sftp()
    
    data_d = ftp.chdir('/mnt/dav/Data')
    cwd=ftp.getcwd()
    path = Path.cwd()
    
    ftp.get("modbusData.db","modbusData.db",callback=None)
    
    conn = sqlite3.connect("modbusData.db")
    query = '''SELECT * from PV'''
    PV_data= pd.read_sql_query(query,conn)
    PV_data['Time']=pd.to_datetime(PV_data['Time'],unit='s')
    
    # PV data 
    PV_data= PV_data.sort_values(by='Time', ascending=True)
    PV_data['Time']=pd.to_datetime(PV_data['Time'],unit='s')
    PV_data = PV_data.set_index('Time')
    PV_data=PV_data.resample('60min').mean()
    PV_data = PV_data.tail(24)
    
    return PV_data

In [29]:
#PV_data =get_modbus()

In [30]:
#PV_data.head()

In [27]:
import pickle
# load the model from disk
path = Path.cwd()
#parentspath = path.parents[0]
filename = 'Skit_learn_model.sav'
loaded_model = pickle.load(open(path/filename, 'rb'))

C:\Users\TrungNguyen\Anaconda3\envs\AI_Model\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\TrungNguyen\Anaconda3\envs\AI_Model\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
